In [ ]:
import math
import random
import time
from ortools.sat.python import cp_model
from readinput import read_multiple_test_cases
file_path = 'input.txt'
test_cases = read_multiple_test_cases(file_path)

In [2]:
def cbus_cp(n, k, distance_matrix):
    model = cp_model.CpModel()

    # Decision variables
    x = {}  # Route variables: x[i][j] = 1 if traveling from i to j
    for i in range(2 * n + 1):
        for j in range(2 * n + 1):
            if i != j:
                x[i, j] = model.NewBoolVar(f'x[{i},{j}]')

    y = [model.NewIntVar(0, k, f'y[{i}]') for i in range(2 * n + 1)]  # Load variables

    u = [model.NewIntVar(0, 2 * n, f'u[{i}]') for i in range(2 * n + 1)]  # Sequence variables

    # Constraints
    # 1. Every node is entered and left exactly once
    for i in range(2 * n + 1):
        model.Add(sum(x[j, i] for j in range(2 * n + 1) if j != i) == 1)
        model.Add(sum(x[i, j] for j in range(2 * n + 1) if j != i) == 1)

    # 2. Load constraints: Maintain capacity
    for i in range(1, n + 1):
        for j in range(1, 2 * n + 1):
            if i != j:
                model.Add(y[j] == y[i] + 1).OnlyEnforceIf(x[i, j])
    
    for i in range(n + 1, 2 * n + 1):
        for j in range(1, 2 * n + 1):
            if i != j:
                model.Add(y[j] == y[i] - 1).OnlyEnforceIf(x[i, j])

    # Subtour elimination using sequence variables
    for i in range(1, 2 * n + 1):
        for j in range(1, 2 * n + 1):
            if i != j:
                model.Add(u[i] + 1 <= u[j]).OnlyEnforceIf(x[i, j])

    # Order constraints: Drop-off after pickup
    for i in range(1, n + 1):
        model.Add(u[i] < u[i + n])

    # Objective: Minimize travel distance
    model.Minimize(
        sum(distance_matrix[i][j] * x[i, j] for i in range(2 * n + 1) for j in range(2 * n + 1) if i != j)
    )

    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f'Min cost: {solver.ObjectiveValue()}')
        route = []
        current = 0
        while len(route) < 2 * n:
            next_point = None
            for j in range(1, 2 * n + 1):
                if j != current and solver.Value(x[current, j]) == 1:
                    next_point = j
                    break
            if next_point is None:
                break
            route.append(next_point)
            current = next_point
        print('Optimal route:', ' '.join(map(str, route)))
    else:
        print('No solution found.')

In [3]:
for idx, (n, k, distance_matrix) in enumerate(test_cases, 1):
    print(f"Test case {idx}:")
    print(f"n = {n}, k = {k}")
    print(f"Cost matrix:")
    for row in distance_matrix:
        print(row)
    print()
    start_time = time.time()
    cbus_cp(n, k, distance_matrix)
    end_time = time.time()
    print(f"Calculation time: {end_time - start_time:.2f} seconds")
    print()

Test case 1:
n = 3, k = 2
Cost matrix:
[0, 8, 5, 1, 10, 5, 9]
[9, 0, 5, 6, 6, 2, 8]
[2, 2, 0, 3, 8, 7, 2]
[5, 3, 4, 0, 3, 2, 7]
[9, 6, 8, 7, 0, 9, 10]
[3, 8, 10, 6, 5, 0, 2]
[3, 4, 4, 5, 2, 2, 0]

Min cost: 25.0
Optimal route: 3 1 4 2 6 5
Calculation time: 0.04 seconds

Test case 2:
n = 5, k = 3
Cost matrix:
[0, 2, 3, 4, 2, 3, 4, 5, 3, 8, 7]
[5, 0, 1, 4, 6, 1, 4, 2, 9, 8, 2]
[1, 2, 0, 4, 2, 3, 4, 5, 3, 6, 2]
[2, 2, 3, 0, 2, 3, 4, 5, 3, 8, 4]
[5, 2, 3, 4, 0, 3, 4, 5, 3, 9, 7]
[1, 2, 3, 1, 2, 0, 4, 2, 2, 8, 2]
[2, 2, 3, 4, 2, 3, 0, 5, 3, 2, 7]
[3, 2, 3, 6, 2, 3, 4, 0, 3, 1, 1]
[5, 2, 3, 4, 2, 3, 4, 5, 0, 8, 7]
[1, 2, 3, 5, 2, 3, 4, 5, 3, 0, 9]
[8, 2, 3, 4, 2, 3, 4, 5, 3, 2, 0]

Min cost: 24.0
Optimal route: 5 3 2 8 1 7 10 4 6 9
Calculation time: 0.05 seconds

Test case 3:
n = 5, k = 3
Cost matrix:
[0, 5, 8, 11, 12, 8, 3, 3, 7, 5, 5]
[5, 0, 3, 5, 7, 5, 3, 4, 2, 2, 2]
[8, 3, 0, 7, 8, 8, 5, 7, 1, 6, 5]
[11, 5, 7, 0, 1, 5, 9, 8, 6, 5, 6]
[12, 7, 8, 1, 0, 6, 10, 10, 7, 7, 7]
[8, 5, 8, 5, 6, 0